In [4]:
import sys
sys.path.append(r"C:\Users\t-zporter\MLOS\source\Mlos.Python")
from mlos.Optimizers.OptimizationProblem import OptimizationProblem, Objective
from mlos.Spaces import SimpleHypergrid, ContinuousDimension, DiscreteDimension, CategoricalDimension

In [5]:
max_num_waves = 3

benchmark_spaces = []
benchmark_spaces.append(
    SimpleHypergrid(
        name="tpcc",
        dimensions=[
            DiscreteDimension(name="tpcc_scale_factor", min=1, max=10),
            DiscreteDimension(name="terminals", min=1, max=10),
            DiscreteDimension(name="execution_time", min=30, max=3*60),
            DiscreteDimension(name="rate_base", min=1, max=10000),
            CategoricalDimension(name="num_waves", values=[*range(max_num_waves + 1)]),
        ] + [DiscreteDimension(f"weight_{x}", min=1, max=100) for x in range(5)]
    )
)

benchmark_spaces.append(
    SimpleHypergrid(
        name="ycsb",
        dimensions=[
            DiscreteDimension(name="scale_factor", min=100, max=5000),
            DiscreteDimension(name="terminals", min=10, max=200),
            DiscreteDimension(name="execution_time", min=30, max=3*60),
            DiscreteDimension(name="rate_base", min=1000, max=50000),
            CategoricalDimension(name="num_waves", values=[*range(max_num_waves + 1)]),
        ] + [DiscreteDimension(f"weight_{x}", min=1, max=100) for x in range(6)]
    )
)

benchmark_spaces.append(
    SimpleHypergrid(
        name="tatp",
        dimensions=[
            DiscreteDimension(name="scale_factor", min=1, max=50),
            DiscreteDimension(name="terminals", min=10, max=200),
            DiscreteDimension(name="execution_time", min=30, max=3*60),
            DiscreteDimension(name="rate_base", min=1000, max=50000),
            CategoricalDimension(name="num_waves", values=[*range(max_num_waves + 1)]),
        ] + [DiscreteDimension(f"weight_{x}", min=1, max=100) for x in range(7)]
    )
)

wave_spaces=[]
for x in range(max_num_waves):
    wave_spaces.append(
        SimpleHypergrid(
            name=f"rate_wave_{x}",
            dimensions=[
                DiscreteDimension("period", min=2*60, max=10*60), # 1/Hz
                DiscreteDimension("amplitude", min=1, max=2000),
                ContinuousDimension("offset", min=-3.14/2.0, max = 3.14/2) # [-pi/2, pi/2]
            ]
        )
    )

for benchmark_space in benchmark_spaces:
    for x in range(max_num_waves):
        benchmark_space = benchmark_space.join(
            subgrid=wave_spaces[x],
            on_external_dimension=CategoricalDimension(name="num_waves", values=[*range(x + 1, max_num_waves + 1)])
        )
        

full_space = SimpleHypergrid(
    name="input",
    dimensions=[
        CategoricalDimension(name=f"dimension_{x}_enabled", values=[True,False]) for x in range(len(benchmark_spaces))
    ]
)

for x in range(len(benchmark_spaces)):
    full_space = full_space.join(
        subgrid = benchmark_spaces[x],
        on_external_dimension=CategoricalDimension(name=f"dimension_{x}_enabled", values=[True])
    )

In [16]:
max_num_waves = 3

benchmark_spaces = {}
benchmark_spaces["tpcc"] = SimpleHypergrid(
    name="tpcc_config",
    dimensions=[
        DiscreteDimension(name="tpcc_scale_factor", min=1, max=10),
        DiscreteDimension(name="terminals", min=1, max=10),
        DiscreteDimension(name="execution_time", min=30, max=3*60),
        DiscreteDimension(name="rate_base", min=1, max=10000),
        CategoricalDimension(name="num_waves", values=[*range(max_num_waves + 1)]),
    ] + [DiscreteDimension(f"weight_{x}", min=1, max=100) for x in range(5)]
)


benchmark_spaces["ycsb"] = SimpleHypergrid(
    name="ycsb_config",
    dimensions=[
        DiscreteDimension(name="scale_factor", min=100, max=5000),
        DiscreteDimension(name="terminals", min=10, max=200),
        DiscreteDimension(name="execution_time", min=30, max=3*60),
        DiscreteDimension(name="rate_base", min=1000, max=50000),
        CategoricalDimension(name="num_waves", values=[*range(max_num_waves + 1)]),
    ] + [DiscreteDimension(f"weight_{x}", min=1, max=100) for x in range(6)]
)


benchmark_spaces["tatp"] = SimpleHypergrid(
    name="tatp_config",
    dimensions=[
        DiscreteDimension(name="scale_factor", min=1, max=50),
        DiscreteDimension(name="terminals", min=10, max=200),
        DiscreteDimension(name="execution_time", min=30, max=3*60),
        DiscreteDimension(name="rate_base", min=1000, max=50000),
        CategoricalDimension(name="num_waves", values=[*range(max_num_waves + 1)]),
    ] + [DiscreteDimension(f"weight_{x}", min=1, max=100) for x in range(7)]
)


wave_spaces=[]
for x in range(max_num_waves):
    wave_spaces.append(
        SimpleHypergrid(
            name=f"rate_wave_{x}",
            dimensions=[
                DiscreteDimension("period", min=2*60, max=10*60), # 1/Hz
                DiscreteDimension("amplitude", min=1, max=2000),
                ContinuousDimension("offset", min=-3.14/2.0, max = 3.14/2.0) # [-pi/2, pi/2]
            ]
        )
    )

for _,benchmark_space in benchmark_spaces.items():
    for x in range(max_num_waves):
        benchmark_space = benchmark_space.join(
            subgrid=wave_spaces[x],
            on_external_dimension=CategoricalDimension(name="num_waves", values=[*range(x + 1, max_num_waves + 1)])
        )
        

full_space = SimpleHypergrid(
    name="input",
    dimensions=[
        CategoricalDimension(name=f"{benchmark_space_name}_enabled", values=[True,False]) for benchmark_space_name in benchmark_spaces
    ]
)

for benchmark_space_name, benchmark_space in benchmark_spaces.items():
    full_space = full_space.join(
        subgrid = benchmark_space,
        on_external_dimension=CategoricalDimension(name=f"{benchmark_space_name}_enabled", values=[True])
    )

print(full_space)

  Name: input
  Dimensions:
    tpcc_enabled: {True, False}
    ycsb_enabled: {True, False}
    tatp_enabled: {True, False}

  IF tpcc_enabled IN {True} THEN (
    Name: tpcc_config
    Dimensions:
      tpcc_scale_factor: {1, 2, ... , 10}
      terminals: {1, 2, ... , 10}
      execution_time: {30, 31, ... , 180}
      rate_base: {1, 2, ... , 10000}
      num_waves: {0, 1, 2, ...}
      weight_0: {1, 2, ... , 100}
      weight_1: {1, 2, ... , 100}
      weight_2: {1, 2, ... , 100}
      weight_3: {1, 2, ... , 100}
      weight_4: {1, 2, ... , 100}

    IF num_waves IN {1, 2, 3} THEN (
      Name: rate_wave_0
      Dimensions:
        period: {120, 121, ... , 600}
        amplitude: {1, 2, ... , 2000}
        offset: [-1.57, 1.57]
    )

    IF num_waves IN {3} THEN (
      Name: rate_wave_2
      Dimensions:
        period: {120, 121, ... , 600}
        amplitude: {1, 2, ... , 2000}
        offset: [-1.57, 1.57]
    )

    IF num_waves IN {2, 3} THEN (
      Name: rate_wave_1
      Di

In [12]:
len(full_space.dimensions)

45

In [13]:
datapoint = full_space.random()
datapoint

{
  "dimension_0_enabled": true,
  "dimension_1_enabled": false,
  "dimension_2_enabled": false,
  "tpcc.tpcc_scale_factor": 8,
  "tpcc.terminals": 9,
  "tpcc.execution_time": 139,
  "tpcc.rate_base": 5582,
  "tpcc.num_waves": 0,
  "tpcc.weight_0": 10,
  "tpcc.weight_1": 2,
  "tpcc.weight_2": 99,
  "tpcc.weight_3": 4,
  "tpcc.weight_4": 71
}

In [7]:
oltp_dict = {}
for key,value in datapoint:
    entries = key.split(".")
    if len(entries) <= 1:
        continue
    # len(entries) >1, thus it is formatted as run_name.value = x
    run_name = entries[0]
    
    if not run_name in oltp_dict:
        oltp_dict[run_name] = {}
        oltp_dict[run_name]["run_type"] = run_name
        oltp_dict[run_name]["weights"] = []
        oltp_dict[run_name]["rate_waves"] = []
    
    if not (entries[1].startswith("rate_wave") or entries[1].startswith("weight")):
        oltp_dict[run_name][entries[1]] = value
        continue
    
    if entries[1].startswith("weight"):
        oltp_dict[run_name]["weights"].append(value)
        continue
    
    if entries[1].startswith("rate_wave"):
        if entries[2] == "period":
            oltp_dict[run_name]["rate_waves"].insert(0,{}) # push to front of array
            oltp_dict[run_name]["rate_waves"][0]["period"] = value
        if entries[2] == "amplitude":
            oltp_dict[run_name]["rate_waves"][0]["amplitude"] = value
        if entries[2] == "offset":
            oltp_dict[run_name]["rate_waves"][0]["offset"] = value
        continue
    
    assert False
    # TODO: It should not be possible to get here. Throw error. 
    
oltp_dict

{'tpcc': {'run_type': 'tpcc',
  'weights': [23, 40, 62, 44, 63],
  'rate_waves': [{'period': 201,
    'amplitude': 1400,
    'offset': 1.4436046637268765},
   {'period': 477, 'amplitude': 950, 'offset': 0.8388531801478425}],
  'tpcc_scale_factor': 10,
  'terminals': 7,
  'execution_time': 48,
  'rate_base': 5650,
  'num_waves': 2},
 'ycsb': {'run_type': 'ycsb',
  'weights': [27, 66, 4, 27, 72, 62],
  'rate_waves': [{'period': 297,
    'amplitude': 422,
    'offset': -0.7302164633172058},
   {'period': 443, 'amplitude': 396, 'offset': 0.8683077973602142},
   {'period': 548, 'amplitude': 807, 'offset': 1.5612126933519892}],
  'scale_factor': 3853,
  'terminals': 176,
  'execution_time': 62,
  'rate_base': 25655,
  'num_waves': 3}}